FIXME: change year -> time

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from ddf_utils.str import to_concept_id

In [3]:
data = pd.read_csv('../source/Credit Suisse Wealth Report - Dataset - v1 - Data Source.csv')

In [29]:
synonyms = pd.read_csv('../../../ddf--open_numbers/ddf--synonyms--geo.csv', dtype=str).set_index('synonym')['geo'].to_dict()

In [31]:
data 

,Year,Country,Adults (Thousands),Mean Wealth per Adult,Median Wealth per Adult,Adults % under 10000,Adults % between 10000-100k,Adults % between 100k-1M,Adults % over 1M,Total,Gini %
0,2020,afg,18356,1744,734,97.6,2.4,0.1,0.0,100,72.8
1,2020,alb,2187,30524,15363,41.0,54.2,4.7,0.1,100,68.2
2,2020,dza,27620,8871,2302,87.0,11.7,1.2,0.1,100,84.8
3,2020,ago,14339,3529,1131,93.5,6.2,0.2,0.0,100,80.6
4,2020,arg,30799,7224,2157,88.2,11.2,0.6,0.0,100,81.2
...,...,...,...,...,...,...,...,...,...,...,...
1542,2012,vnm,60431,4652,2257,89.6,10.3,0.2,0.0,100,65.9
1543,2012,pse,1738,8856,4320,72.5,26.9,0.6,0.0,100,65.9
1544,2012,yem,11732,4731,2228,89.7,10.1,0.2,0.0,100,66.9
1545,2012,zmb,5991,1742,539,97.0,3.0,0.0,0.0,100,74.1


In [33]:
data['Country'].hasnans

False

In [5]:
def get_mapping(x):
    try:
        return synonyms[x]
    except KeyError:
        print(x)
        return to_concept_id(x)

data['Country'] = data['Country'].map(get_mapping)

British Caribbean
Dutch Caribbean
French Caribbean
Melanesia
Polynesia


In [6]:
df = data.set_index(['Country', 'Year'])

In [7]:
df.columns

Index([' Adults (Thousands)', ' Mean Wealth per Adult',
       ' Median Wealth per Adult', ' Adults % under 10000',
       ' Adults % between 10000-100k', ' Adults % between 100k-1M',
       ' Adults % over 1M', ' Total', ' Gini %'],
      dtype='object')

In [8]:
indicators1 = df[[' Adults (Thousands)', ' Mean Wealth per Adult',
                  ' Median Wealth per Adult', ' Gini %']].copy()

In [9]:
indicators1.index.names = ['geo', 'time']
indicators1 = indicators1.groupby(level=['geo']).apply(lambda x: x)
for c in indicators1.columns:
    ser = indicators1[c]
    concept_id = to_concept_id(c)
    ser.name = concept_id
    # print(ser.head(1))
    ser.to_csv(f'../../ddf--datapoints--{concept_id}--by--geo--time.csv')

In [34]:
indicators2 = df[[' Adults % under 10000',
                  ' Adults % between 10000-100k', ' Adults % between 100k-1M',
                  ' Adults % over 1M']].copy()
indicators2.index.names = ['geo', 'time']
m = {
    ' Adults % under 10000': 'under_10k',
    ' Adults % between 10000-100k': 'between_10k_100k',
    ' Adults % between 100k-1M': 'between_100k_1m',
    ' Adults % over 1M': 'over_1m'
}
indicators2.columns = indicators2.columns.map(m)
indicators2.columns.name = 'wealth_group'
indicators2 = indicators2.stack()

In [48]:
indicators2.to_frame().reset_index()

,geo,time,wealth_group,adult_percentage
0,afg,2020,under_10k,97.6
1,afg,2020,between_10k_100k,2.4
2,afg,2020,between_100k_1m,0.1
3,afg,2020,over_1m,0.0
4,alb,2020,under_10k,41.0
...,...,...,...,...
6182,zmb,2012,over_1m,0.0
6183,zwe,2012,under_10k,95.1
6184,zwe,2012,between_10k_100k,4.8
6185,zwe,2012,between_100k_1m,0.2


In [39]:
indicators2.name = 'adult_percentage'
# indicators2.index.names = ['geo', 'time', 'wealth_group']
indicators2.to_csv('../../ddf--datapoints--adult_percentage--by--geo--time--wealth_group.csv')

In [56]:
adults = df[' Adults (Thousands)']
grps = indicators2.to_frame().groupby(['geo', 'time'])
res = []

def get_number(i):
    return adults.loc[i]

for k, ser in grps:
    # print('i')
    # print(k, ser)
    res.append(ser * get_number(k))

In [57]:
res1 = pd.concat(res)

In [59]:
res1 

adult_percentage
geo time wealth_group                      
abw 2018 under_10k                   3160.0
         between_10k_100k            3950.0
         between_100k_1m              766.3
         over_1m                       23.7
    2019 under_10k                   2400.0
...                                     ...
zwe 2019 over_1m                        0.0
    2020 under_10k                 615773.4
         between_10k_100k           88575.0
         between_100k_1m             4251.6
         over_1m                        0.0

[6187 rows x 1 columns]

In [58]:
levels = [10000, 100000, 1000000]
levels_daily = list(map(lambda x: x * 0.03 / 365, levels))

In [ ]:
levels_daily 

[0.821917808219178, 8.219178082191782, 82.1917808219178]

In [60]:
res1 = res1.reset_index()

In [61]:
res1.wealth_group.unique()

array(['under_10k', 'between_10k_100k', 'between_100k_1m', 'over_1m'],
      dtype=object)

In [62]:
m = {
    'under_10k': 'level1',
    'between_10k_100k': 'level2',
    'between_100k_1m': 'level3',
    'over_1m': 'level4',
}

In [63]:
res2 = res1.copy()
res2['income_level'] = res2['wealth_group'].map(lambda x: m[x])
res2 = res2.drop(columns=['wealth_group'])

In [64]:
res2.columns = ['geo', 'time', 'adult_percentage', 'income_level']

In [65]:
# new_idx = pd.MultiIndex.from_product([res2.country.unique(),
#                                       res2.year.unique(),
#                                       res2.income_level.unique()])

res2 = res2.set_index(['geo', 'time', 'income_level'])
# res2 = res2.reindex(new_idx)

In [66]:
res2['adult_percentage'].hasnans

False

In [ ]:
# res2 = res2['adult_percentage'].fillna(0)

In [67]:
res2.name = 'population'

In [68]:
res2 = np.floor(res2)
res2.astype(int).sort_index().to_csv('../../ddf--datapoints--population--by--geo--time--income_level.csv')

In [69]:
res3 = indicators2.reset_index()

In [70]:
res3['wealth_group'] = res3['wealth_group'].map(lambda x: m[x])

In [71]:
res3.columns

Index(['geo', 'time', 'wealth_group', 'adult_percentage'], dtype='object')

In [72]:
res3.columns = ['geo', 'time', 'income_level', 'adult_percentage']

In [73]:
res3.to_csv('../../ddf--datapoints--adult_percentage--by--geo--time--income_level.csv', index=False)

In [ ]:
# TODO: add some entities

In [74]:
names1=  ['Adults (Thousands)', 'Mean Wealth per Adult',
                  'Median Wealth per Adult', ' Gini %']
cdf = pd.DataFrame(names1)

In [75]:
cdf.columns = ['name']
cdf["concept"] = cdf['name'].map(to_concept_id)
cdf['concept_type'] = 'measure'

In [76]:
cdf

,name,concept,concept_type
0,Adults (Thousands),adults_thousands,measure
1,Mean Wealth per Adult,mean_wealth_per_adult,measure
2,Median Wealth per Adult,median_wealth_per_adult,measure
3,Gini %,gini_pct,measure


In [77]:
cdf[['concept', 'concept_type', 'name']].to_csv('../../ddf--concepts.csv', index=False)

In [78]:
cdf = pd.read_csv('../../../ddf--open_numbers/ddf--concepts.csv', dtype=str)

In [79]:
cdf2= pd.read_csv('../../ddf--concepts.csv')

In [80]:
cdf2

,concept,concept_type,name
0,adults_thousands,measure,Adults (Thousands)
1,mean_wealth_per_adult,measure,Mean Wealth per Adult
2,median_wealth_per_adult,measure,Median Wealth per Adult
3,gini_pct,measure,Gini %


In [81]:
cdf = cdf.set_index('concept')
cdf2 = cdf2.set_index('concept')

In [82]:
cdf3 = pd.concat([cdf, cdf2])

In [83]:
cdf3.to_csv('../../ddf--concepts.csv')

In [85]:
# FIXME: add entities concepts and population